##Streamlit UI
Create a new file finbot_app.py:

##Explanation:
- @st.cache_resource caches the model to avoid reloading

- We recreate the conversation chain with the fine-tuned model

- Streamlit provides a chat interface with message history

- User inputs are added to conversation history and displayed

- The model generates responses that are displayed in the UI

In [ ]:
%%writefile app.py

import streamlit as st
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model
@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("./finbot_finetuned")
    model = AutoModelForCausalLM.from_pretrained(
        "./finbot_finetuned",
        device_map="auto",
        torch_dtype=torch.float16,
    )
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0.7,
    )
    return pipe

pipe = load_model()

# Set up LangChain
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=pipe, memory=memory)

# Streamlit UI
st.title("FinBot - Financial Literacy Assistant")
st.write("Ask me anything about personal finance!")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("What financial question do you have?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Get bot response
    response = conversation.predict(input=prompt)

    # Display bot response
    with st.chat_message("assistant"):
        st.markdown(response)
    st.session_state.messages.append({"role": "assistant", "content": response})

#Running the Application
running the Streamlit app from Colab:

##Explanation:
- Since Colab doesn't directly expose web apps, we use localtunnel

- This will give you a public URL to access your Streamlit app

- The app runs on port 8501 (default Streamlit port)

In [ ]:
!npm install localtunnel
!streamlit run finbot_app.py &>/content/logs.txt &
!npx localtunnel --port 8501